In [ ]:
open('cafe.txt', 'w', encoding='utf8').write('café')

4

In [ ]:
open('cafe.txt').read()

'cafe'

In [ ]:
import sys, locale

expressions = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
        """
my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'utf-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


In [ ]:
from unicodedata import normalize

s1 = open('cafe.txt').read()
s2 = open('cafe.txt').read()

len(normalize('NFC',  s1)), len(normalize('NFC', s2))

(4, 4)

In [ ]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [ ]:
normalize('NFC', s1), normalize('NFC', s2)

('café', 'café')

In [ ]:
normalize('NFD', s1), normalize('NFD', s2)

('café', 'café')

In [ ]:
normalize('NFC', s1) == normalize('NFC', s2)

True

In [ ]:
normalize('NFD', s1) == normalize('NFD', s2)

True

In [ ]:
from unicodedata import normalize, name
ohm = '\u2126'
name(ohm)

'OHM SIGN'

In [ ]:
ohm_c = normalize('NFC', ohm)
name(ohm_c)

'GREEK CAPITAL LETTER OMEGA'

In [ ]:
ohm, ohm_c, ohm == ohm_c

('Ω', 'Ω', False)

In [ ]:
normalize('NFC', ohm) == normalize('NFC', ohm_c)

True

In [ ]:
eszett = 'ß'
name(eszett)

'LATIN SMALL LETTER SHARP S'

In [ ]:
eszett_cf = eszett.casefold()
eszett_cf2 = eszett.lower()
eszett, eszett_cf, eszett_cf2

('ß', 'ss', 'ß')

In [ ]:
s1 = 'café'
s2 = 'cafe\u0301'
s1 == s2

False

In [ ]:
from unicodedata import normalize

def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() ==
            normalize('NFC', str2).casefold())

nfc_equal(s1, s2)

True

In [ ]:
nfc_equal('A', 'a')

False

In [ ]:
s3 = 'Straße'
s4 = 'strasse'
s3 == s4

False

In [ ]:
nfc_equal(s3, s4)

False

In [ ]:
fold_equal(s3, s4)

True

In [ ]:
fold_equal(s1, s2)

True

In [ ]:
fold_equal('A', 'a')

True

In [ ]:
import unicodedata
import string

def shave_marks(txt):
  norm_txt = unicodedata.normalize('NFD', txt)
  shaved = ''.join(c for c in norm_txt
                   if not unicodedata.combining(c))
  return unicodedata.normalize('NFC', shaved)

In [ ]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
shave_marks(order)

'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

In [ ]:
Greek = 'Ζέφυρος, Zéfiro'
shave_marks(Greek)

'Ζεφυρος, Zefiro'

In [ ]:
def shave_marks_latin(txt):
  norm_txt = unicodedata.normalize('NFD', txt)

  latin_base = False
  keepers = []

  for c in norm_txt:
    if unicodedata.combining(c) and latin_base:
      continue
    keepers.append(c)

    if not unicodedata.combining(c):
      latin_base = c in string.ascii_letters

  shaved = ''.join(keepers)
  return unicodedata.normalize('NFC', shaved)

In [ ]:
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",
                           """'f"*^<''""---~>""")

multi_map = str.maketrans({
  '€': '<euro>',
  '…': '...',
  'Œ': 'OE',
  '™': '(TM)',
  'œ': 'oe',
  '‰': '<per mille>',
  '‡': '**',
})

multi_map.update(single_map)

def dewinize(txt):
  return txt.translate(multi_map)

def asciize(txt):
  no_marks = shave_marks_latin(dewinize(txt))
  no_marks = no_marks.replace('ß', 'ss')
  return unicodedata.normalize('NFKC', no_marks)


In [ ]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
dewinize(order)

'"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."'

In [ ]:
asciize(order)

'"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."'

In [ ]:
fruits = ['caju', 'atemoia', 'cajá', 'acaí', 'acerola']
sorted(fruits)

['acaí', 'acerola', 'atemoia', 'caju', 'cajá']

In [ ]:
!pip install pyuca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.4 MB/s eta 0:00:00


In [ ]:
import pyuca

coll = pyuca.Collator()
sorted_fruits = sorted(fruits, key=coll.sort_key)
sorted_fruits

['acaí', 'acerola', 'atemoia', 'cajá', 'caju']

In [ ]:
import unicodedata
import re

re_digit = re.compile(r'\d')
sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
  print('U+%04x' % ord(char),
        char.center(6),
        're_dig' if re_digit.match(char) else '-',
        'isdig' if char.isdigit() else '-',
        'isnum' if char.isnumeric() else '-',
        format(unicodedata.numeric(char), '5.2f'),
        unicodedata.name(char),
        sep='\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


In [ ]:
import re

re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"
            " as 1729 = 1³ + 12³ = 9³ + 10³.")

text_bytes = text_str.encode('utf_8')

print('Text', repr(text_str), sep='\n  ')
print('Numbers')
print(' str :', re_numbers_str.findall(text_str))
print(' bytes:', re_numbers_bytes.findall(text_bytes))
print('Words')
print(' str :', re_words_str.findall(text_str))
print(' bytes:', re_words_bytes.findall(text_bytes))

Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
 str : ['௧௭௨௯', '1729', '1', '12', '9', '10']
 bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
 str : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
 bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


In [ ]:
import os

os.listdir('.')

['.config', 'dummy', 'cafe.txt', 'sample_data']

In [ ]:
os.listdir(b'.')

[b'.config', b'dummy', b'cafe.txt', b'sample_data']

In [71]:
import os

# 디렉터리 경로 설정
directory = './fluent_python'

# 디렉터리가 없으면 생성
if not os.path.exists(directory):
    os.makedirs(directory)

# 첫 번째 파일 생성 - digita-of-n.txt
file_path1 = os.path.join(directory, 'digita-of-π.txt')
with open(file_path1, 'w') as file:
    file.write('')  # 빈 파일 생성


In [72]:
os.listdir('./fluent_python')

['digita-of-π.txt']

In [73]:
os.listdir(b'./fluent_python')

[b'digita-of-\xcf\x80.txt']

In [ ]:
pi_name_bytes = os.listdir(b'.')[1]
pi_name_str = pi_name_bytes.decode('ascii', 'surrogateescape')
pi_name_str

'dummy'